In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PsRxZWijDRPuYfAUghaSALVzvm9UNoNs6llpbk5q3y8i',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'gestureidentification-donotdelete-pr-bpwz2af7ki1ztl'
object_key = 'Dataset-20221112T112148Z-001.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
filepaths = unzip.namelist()
for path in filepaths:
    unzip.extract(path)

In [5]:
train_datagen = ImageDataGenerator(rescale=1.0/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
X_train = train_datagen.flow_from_directory('Dataset/train',target_size=(64,64),batch_size=32,class_mode = 'categorical',color_mode='grayscale')
x_test = train_datagen.flow_from_directory('Dataset/test',target_size=(64,64),batch_size=32,class_mode = 'categorical',color_mode='grayscale')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [7]:
#model building
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
classifier = Sequential()


In [8]:
classifier.add(Conv2D(32,(3,3),input_shape =(64,64,1),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=6,activation='softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [9]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
classifier.fit_generator(
    generator=X_train,steps_per_epoch=len(X_train),
    epochs=20,validation_data=x_test,validation_steps=len(x_test)
)

/tmp/wsuser/ipykernel_164/1477240979.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
19/19 [==============================] - 4s 161ms/step - loss: 1.7552 - accuracy: 0.2626 - val_loss: 1.6149 - val_accuracy: 0.3333
Epoch 2/20
19/19 [==============================] - 3s 160ms/step - loss: 1.3032 - accuracy: 0.5589 - val_loss: 1.0376 - val_accuracy: 0.5333
Epoch 3/20
19/19 [==============================] - 3s 144ms/step - loss: 0.8389 - accuracy: 0.6919 - val_loss: 0.8143 - val_accuracy: 0.6667
Epoch 4/20
19/19 [==============================] - 3s 145ms/step - loss: 0.6669 - accuracy: 0.7104 - val_loss: 0.5315 - val_accuracy: 0.8667
Epoch 5/20
19/19 [==============================] - 3s 146ms/step - loss: 0.5509 - accuracy: 0.7694 - val_loss: 0.4404 - val_accuracy: 0.9000
Epoch 6/20
19/19 [==============================] - 3s 146ms/step - loss: 0.4436 - accuracy: 0.8148 - val_loss: 0.5756 - val_accuracy: 0.8000
Epoch 7/20
19/19 [==============================] - 3s 151ms/step - loss: 0.4051 - accuracy: 0.8367 - val_loss: 0.4291 - val_accuracy: 0.9000
Epoch 

In [11]:
classifier.save('gesture.h5')
model_json=classifier.to_json()
with open("model-bw.json",'w')as json_file:
  json_file.write(model_json)

In [ ]:
!tar -zcvf model.tgz gesture.h5

In [19]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array
model = load_model("gesture.h5")

In [41]:
img = load_img('/content/drive/MyDrive/IBM/test/2/1.jpg',grayscale=True,
                     target_size=(64,64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)
pred

array([[0., 0., 1., 0., 0., 0.]], dtype=float32)

In [21]:
from ibm_watson_machine_learning import APIClient
wml_credentials ={
        "url":"https://us-south.ml.cloud.ibm.com",
        "apikey":"o9dTlYeIrolX2yzbm3pD1eNKu--FJXoZmgl4alDutFkq"
    }
client = APIClient(wml_credentials)

In [23]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [24]:
space_uid = guid_from_space_name(client,'gestureidentification')
print(space_uid)

05ca3c80-b748-421b-8fae-010bef086c0a


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [17]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [20]:
model_details=client.repository.store_model(model='model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:'CNN',
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_id(model_details)
model_id

'faeb0c29-65a4-4bf8-854b-388f14e02613'

In [26]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'